**Softmax**

$S(y_i) = \frac{e^{y_i}}{\sum e^{y_i}}$

In [4]:
import torch
import torch.nn as nn
import numpy as np

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)


x = np.array([2.0, 1.0, 0.1])
output = softmax(x)
print("softmax output", output)

softmax output [0.65900114 0.24243297 0.09856589]
1.0


In [13]:
# dans PyTorch

x = torch.tensor([2.0, 1.0, 0.1])
output = torch.softmax(x, dim=0)  # Computes along the first axis
print(output)

tensor([0.6590, 0.2424, 0.0986])


**Cross Entropy**

$D(\hat{Y}, Y) = \frac{-1}{N}\cdot \sum Y_i \cdot \log{\hat{Y}_i}$

In [18]:
#Avec Numpy
def cross_entropy(actual, predicted):
    loss = -np.sum(actual * np.log(predicted))
    return loss #/ float(np.array(predicted).shape[0])  #Il est possible de diviser par N pour normaliser


# Y doit être une "one-hot encoded" :
# Si classe 0 : [1,0,0]  -> rouge
# Si classe 1 : [0,1,0]  -> bleu
# Si classe 2 : [0,0,1]  -> vert (par exemple)
Y = np.array([1, 0, 0])

#y_pred has propabilities
y_pred_good = [0.7, 0.2, 0.1]  #Bonne prédiction
y_pred_bad =  [0.1, 0.3, 0.6]  #Mauvaise prédiction

l1 = cross_entropy(Y, y_pred_good)
l2 = cross_entropy(Y, y_pred_bad)

print(f"Loss1 numpy : {l1:.4f}")
print(f"Loss2 numoy : {l2:.4f}")

Loss1 numpy : 0.3567
Loss2 numoy : 2.3026


In [21]:
#Avec PyTorch

loss = nn.CrossEntropyLoss()
#nn.CrossEntropyLoss() applique déjà nn.LogSoftmax et nn.NLLLoss (negative log likelihood loss)
# Y doit être un label et non un One-Hot -> Classe : différents outputs possibles
# Y_pred ne doit pas avoir de softmax

Y = torch.tensor([0])  #[0] ou [1]
# nsamples x nclasses = 1x3
y_pred_good = torch.tensor([[2.0, 1.0, 0.1]])  #raw value -> no softmax apply
y_pred_bad = torch.tensor([[0.5, 2.0, 0.3]])   #[[classe0, classe1, classe2]]

l1 = loss(y_pred_good, Y)
l2 = loss(y_pred_bad, Y)

print(f"Loss1 tensor : {l1:.4f}")
print(f"Loss2 tensor : {l2:.4f}")


_, prediction1 = torch.max(y_pred_good, 1)

_, prediction2 = torch.max(y_pred_bad, 1)

print(prediction1)  #Return [0] pour classe 0 : ce qui est bien notre Y
print(prediction2)  #Return [1] pour classe 1 : ce qui n'est pas notre Y

Loss1 tensor : 0.4170
Loss2 tensor : 1.8406
tensor([0])
tensor([1])


**Les classes**

Les classes permettent d'encoder de l'information. Par exemple, on peut encoder des couleurs ainsi:

- Classe 0 : [1,0,0] $\rightarrow$ rouge
- Classe 1 : [0,1,0] $\rightarrow$ bleu
- Classe 2 : [0,0,1] $\rightarrow$ vert

Prenons un exemple avec 3 samples et 3 classes:

In [22]:
loss = nn.CrossEntropyLoss()

# 3 samples donc 3 classes possibles
Y = torch.tensor([2, 0, 1])  #3 classes labels -> La première bonne label est la 2, puis la deuxième est la 0 et finalement la 1

#nsample x nclasses = 3x3
#Ici, on vient analyser plusieurs samples de classes à prédire

y_pred_good = torch.tensor([[0.1,1.0,2.1], [2.0,1.0,0.1], [0.1,3.0,0.1]])   #[[prob_classe 0, prob_classe1, prob_classe2], [...], [...]]
y_pred_bad = torch.tensor([[2.1, 1.0, 0.1], [0.1, 1.0, 2.1], [0.1, 3.0, 0.1]])

_, prediction1 = torch.max(y_pred_good, 1)

_, prediction2 = torch.max(y_pred_bad, 1)

print(prediction1)  #Retourne [2,0,1] -> Toutes les classes ont bien étés prédits
print(prediction2)  #Retourne [0,2,1] -> Aucune classe prédit

tensor([2, 0, 1])
tensor([0, 2, 1])


**Code d'activation**
Les fonctions d'activations les plus utilisées sont :
- Step function
- Sigmoid
- Tanh
- ReLu
- Leaky ReLu
- Softmax

Voici 2 méthodes différentes qui donne le même résultat

In [23]:
class NeuralNet(nn.Module):

    def __init__(self, input_size, hidden_size):
        super(NeuralNet, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)  # First linear layer
        self.relu = nn.ReLU  # Second layer -> ReLu
        self.linear2 = nn.Linear(hidden_size, 1)  #Third layer -> Layer
        self.sigmoid = nn.Sigmoid()  #Final layer -> sigmoid
#On constate que la layer final est sigmoid. Elle fonctionne bien lorsque la couche précédente
#possède un output .... c'est pourquoi nous avons hidden_size=1 pour la third layer

    def foward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out


class NeuralNet(nn.Module):

    def __init__(self, input_size, hidden_size):
        super(NeuralNet, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, 1)

    def foward(self, x):
        out = torch.relu(self.linear1(x))
        out = torch.sigmoid(self.linear2(out))

**Feed Foward Net**

In [4]:
import torch
import torch.nn
import torchvision
import torchvision.transforms as transforms
import matplotlib_inline
import matplotlib.pyplot as plt

# Compute on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# hyper parameter 
input_size = 784  #image size 28x28
hidden_size = 100  #choix arbitraire
num_classes = 10
num_epoch = 2  #pour diminuer temps de training
batch_size = 100
learning_rate = 0.001

#MNIST -> data
#train_dataset = torchvision.datasets.MNIST(root="", train=True, transform=transforms.ToTensor(), download=True)
#test_dataset = torchvision.datasets.MNIST(root="", train=False, transform=transforms.ToTensor())

#train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

#examples = iter(train_loader)
#samples, labels = examples.next()

print(samples.shape, labels.shape)

torch.Size([100, 1, 28, 28]) torch.Size([100])
